#### IMPORTS

In [2]:
import os
import numpy as np
import pandas as pd

def find_r_installation():
    possible_paths = []

    # Add standard paths for Windows
    if os.getenv('ProgramFiles'):
        possible_paths.append(os.path.join(os.getenv('ProgramFiles'), 'R'))
    if os.getenv('ProgramFiles(x86)'):
        possible_paths.append(os.path.join(os.getenv('ProgramFiles(x86)'), 'R'))
    if os.getenv('LOCALAPPDATA'):
        possible_paths.append(os.path.join(os.getenv('LOCALAPPDATA'), 'Programs', 'R'))
    
    # Add standard path for Unix-like systems
    if os.getenv('HOME'):
        possible_paths.append(os.path.join(os.getenv('HOME'), 'R'))

    for path in possible_paths:
        if os.path.exists(path):
            for root, dirs, files in os.walk(path):
                if 'bin' in dirs:
                    if 'R.exe' in os.listdir(os.path.join(root, 'bin')):  # Check specifically for Windows
                        return os.path.join(root)
                    if 'R' in os.listdir(os.path.join(root, 'bin')):  # Check for Unix-like systems
                        return os.path.join(root)

    return None

r_path = find_r_installation()
if r_path:
    print(f"R is installed at: {r_path}")
else:
    print("R installation not found.")

os.environ['R_HOME'] = r_path

R is installed at: C:\Users\maceo.valente\AppData\Local\Programs\R\R-4.3.2


In [9]:
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr
from rpy2.robjects import r

pandas2ri.activate()

Generating weatherFile from EPW

In [4]:
# Lire le fichier EPW en sautant les 8 premières lignes de métadonnées
epw_file_path = 'FRA_PR_Orange.Caritat.AB.075790_TMYx.epw'
epw_data = pd.read_csv(epw_file_path, skiprows=8, header=None)

# Créer le DataFrame weatherFile avec les colonnes de base
date_range = pd.date_range(start='2022-01-01', end='2022-12-31')
weatherFile = pd.DataFrame({
    'file': ['weatherFile_TEST'] * len(date_range),
    'year': [2022] * len(date_range),
    'month': date_range.month,
    'day': date_range.day,
    'doy': date_range.dayofyear
})

# Initialiser les colonnes supplémentaires
weatherFile['temp_min'] = np.nan
weatherFile['temp_max'] = np.nan
weatherFile['Radiation'] = np.nan
weatherFile['Etp'] = -999.9
weatherFile['Rain'] = np.nan
weatherFile['Wind'] = np.nan
weatherFile['Tpm'] = -999.9
weatherFile['Co2'] = 412

# Calculer les valeurs journalières pour chaque colonne
for i, date in enumerate(date_range):
    start_hour = i * 24
    end_hour = start_hour + 24
    
    daily_data = epw_data.iloc[start_hour:end_hour]
    
    weatherFile.at[i, 'temp_min'] = daily_data[6].min()  # column G
    weatherFile.at[i, 'temp_max'] = daily_data[6].max()  # column G
    weatherFile.at[i, 'Radiation'] = daily_data[13].sum() * 0.0036  # column N, convert Wh/m2 to MJ/m2
    weatherFile.at[i, 'Rain'] = daily_data[33].sum()  # column AH
    weatherFile.at[i, 'Wind'] = daily_data[21].mean()  # column V

weatherFile = weatherFile.round(2)
# Afficher les premières lignes du DataFrame pour vérification
print(weatherFile.head())

# Exporter le DataFrame en fichier CSV
weatherFile.to_csv('R_related/weatherFilesSource/weatherFile_TEST.2022', sep=' ', index=False, header = False)

weatherFile2 = weatherFile.copy()
weatherFile2['year'] = 2023

weatherFile2.to_csv('R_related/weatherFilesSource/weatherFile_TEST.2023', sep=' ', index=False, header = False)



               file  year  month  day  doy  temp_min  temp_max  Radiation  \
0  weatherFile_TEST  2022      1    1    1       6.3      12.5       3.01   
1  weatherFile_TEST  2022      1    2    2       6.0      11.0       2.73   
2  weatherFile_TEST  2022      1    3    3       3.3      12.0       4.36   
3  weatherFile_TEST  2022      1    4    4       6.0      17.0       6.60   
4  weatherFile_TEST  2022      1    5    5       3.0      15.0       7.04   

     Etp  Rain  Wind    Tpm  Co2  
0 -999.9  11.4  4.19 -999.9  412  
1 -999.9   2.3  0.93 -999.9  412  
2 -999.9   0.1  1.38 -999.9  412  
3 -999.9   0.0  3.30 -999.9  412  
4 -999.9   0.0  1.14 -999.9  412  


In [6]:
# import os
# import numpy as np
# import pandas as pd
# START_DATE = pd.Timestamp('2023-01-01')

# START_DATE_PLUS_YEAR = START_DATE + pd.DateOffset(years=1, days = -1)

# date_range = pd.date_range(start=START_DATE, end=START_DATE_PLUS_YEAR)

# print((START_DATE + pd.DateOffset(years=1)).year)

# print('R_related/weatherFilesSource/weatherFile_TEST.'+str(START_DATE.year + 1))

2024
R_related/weatherFilesSource/weatherFile_TEST.2024


In [7]:

# Charger le fichier R contenant la fonction

f_change_scenar = ro.globalenv['change_scenar']


# # Fonction Python pour appeler la fonction R
# def call_r_function(param1, param2, param3):
#     result = my_r_function(param1, param2, param3)
#     return result[0]

# # Exemple d'utilisation
# params = [1, 2, 3]
# result = call_r_function(*params)
# print(f"Result from R function: {result}")


In [ ]:
r.source("Rfunctions.R")

f_settings = ro.globalenv['settings']
f_settings("/R_related")

Setting working directory to: C:/Users/maceo.valente/Documents/Automatisation/TEST_MVA/TSAgriPV/R_related 
Get working dir :  C:/Users/maceo.valente/Documents/Automatisation/TEST_MVA/TSAgriPV/R_related 
Javastics path :  C:/Users/maceo.valente/Documents/Automatisation/TEST_MVA/TSAgriPV/JavaSTICS/JavaStics.exe 
Workspace path :  C:/Users/maceo.valente/Documents/Automatisation/TEST_MVA/TSAgriPV/R_related/Rproject 
Le dossier existe déjà: Rproject/weatherFiles/ 


<rpy2.rinterface_lib.sexp.NULLType object at 0x0000019AADBF8A48> [RTYPES.NILSXP]

In [11]:
# Définir le chemin du script R
script_path = 'simu_agro.R'

# Lire le script R
with open(script_path, 'r') as file:
    script_content = file.read()

# Exécuter le script R
ro.r(script_content)
ro.r('source("simu_agro.R")')
my_df = ro.globalenv['resultsSimu2']

v Using stics "modulostics" (exe: "stics_modulo.exe")
[1] "VignePioGre"
[1] "[25/07/24]-[14:29:30] INFO - Files generated :"                                                                                                  
[2] "[25/07/24]-[14:29:30] INFO - \tC:\\Users\\maceo.valente\\Documents\\Automatisation\\TEST_MVA\\TSAgriPV\\R_related\\Rproject\\mod_bVignePioGre.sti"
[3] "[25/07/24]-[14:29:30] INFO - \tC:\\Users\\maceo.valente\\Documents\\Automatisation\\TEST_MVA\\TSAgriPV\\R_related\\Rproject\\modhistory.sti"      
v All usms ran successfully!
[1] 11.91
[1] 14.65057
           V1
1   0.7699422
2   0.7091153
3   0.7508562
4   0.7420789
5   0.7527387
6   0.7428250
7   0.7375776
8   0.7325490
9   0.7262931
10  0.6370107
11  0.7179742
12  0.7060229
13  0.7110771
14  0.7083069
15  0.7089995
16  0.7049743
17  0.7075209
18  0.7179054
19  0.6252294
20  0.7005880
21  0.6454721
22  0.7497704
23  0.7455908
24  0.7324159
25  0.7671579
26  0.7304598
27  0.6320413
28  0.6885355
29  0.7846701
3

In [26]:
# Importer le fichier R contenant le dataframe
ro.r('source("simu_agro.R")')

# Récupérer le dataframe R
yield1 = ro.r['yieldPdsFruitFrais']
yield2 = ro.r['yieldPdsFruitFrais2']

print(yield2/yield1 * 100)

v Using stics "modulostics" (exe: "stics_modulo.exe")
[1] "VignePioGre"
[1] "[25/07/24]-[14:59:09] INFO - Files generated :"                                                                                                  
[2] "[25/07/24]-[14:59:09] INFO - \tC:\\Users\\maceo.valente\\Documents\\Automatisation\\TEST_MVA\\TSAgriPV\\R_related\\Rproject\\mod_bVignePioGre.sti"
[3] "[25/07/24]-[14:59:09] INFO - \tC:\\Users\\maceo.valente\\Documents\\Automatisation\\TEST_MVA\\TSAgriPV\\R_related\\Rproject\\modhistory.sti"      
v All usms ran successfully!
[1] 11.91
[1] 14.65057
v Using stics "modulostics" (exe: "stics_modulo.exe")
[1] "VignePioGre"
[1] "[25/07/24]-[14:59:13] INFO - Files generated :"                                                                                                  
[2] "[25/07/24]-[14:59:13] INFO - \tC:\\Users\\maceo.valente\\Documents\\Automatisation\\TEST_MVA\\TSAgriPV\\R_related\\Rproject\\mod_bVignePioGre.sti"
[3] "[25/07/24]-[14:59:13] INFO - \tC:\\Users\

In [25]:
variables_py = list(variables)

print("Variables créées par le script R:", variables_py)


var = ro.r['yieldPdsFruitFrais']

print(var)

Variables créées par le script R: ['change_scenar', 'fichier', 'generate_ini', 'generate_sols', 'generate_sta', 'generate_tec', 'generate_usm', 'javastics_path', 'printonenv', 'rap_perso', 'resultsSimu1', 'resultsSimu2', 'runs_inf', 'settings', 'workspace_path', 'yieldMaFruit', 'yieldMaFruit2', 'yieldPdsFruitFrais', 'yieldPdsFruitFrais2']
[14.65057]


In [28]:
var2 = ro.r['yieldPdsFruitFrais2']
var2

array([14.5709])